<hr>
<font face="Calibri">

<font size="6"> <b> Flood Depth Estimation with Flood Extent Maps </b> </font> 



<br>
<font size="4"> <b> Part of NASA A.37 Project:</b> Integrating SAR Data for Improved Resilience and Response to Weather-Related Disasters   <br>
<font size="4"> <b> PI:</b>Franz J. Meyer <br>
<font size="3"> Version 0.1.3 - 2020/10/01 <br>
<b>Change Log</b><br>
2020/10/01: <br>
- Feat: Moving away from repetitive argwhere to ndimage.find_objects results in ~6000x faster calculation.
2020/09/30: <br>
- BugFix: The known water mask and input mask was merged using a sum operator instead of a bitwise_or operator.<br> 
2020/09/20:<br>
- BugFix: Added known water download and addition to the mask. This helps to make sure known water bodies are handled as a single object. Removed morphological filters also. <br>
2020/09/17:<br>
- First version. 
<font face="Calibri">
<font size="3"> This notebook provides the processor to generate Flood Depth map using the product generated by <b><font color='rgba(200,0,0,0.2)'>Hyp3 Change Detection-Threshold </font></b> processor. This notebook can be used to generate <b><font color='rgba(200,0,0,0.2)'> Multiple </font></b> FD Products  </font>
<br><br>
<font face="Calibri">
<font size="3"> * Before you start to use the notebook, <b><font color='rgba(200,0,0,0.2)'>Hyp3-generated change detection maps in Geotiff format </font></b> need to be placed in your own data folder  </font>

<br><br>
<font face="Calibri" size="4"><b>Set the modules and libraries for processing </b> </font>

In [1]:
#Setup Environment
import os
import urllib
import numpy as np
import gdal
import json
import glob
from scipy import ndimage
from scipy import interpolate
from scipy import optimize
from importlib import reload
from skimage import morphology
import pylab as pl
import pprint
import concurrent.futures as cf
from IPython.core.debugger import set_trace
#The two lines below are for visually browsing and selecting the DEM. 
import ipywidgets as ui
from IPython.display import display

import warnings

try:
    import cv2
except:
    !pip install --user opencv-python
    import cv2
try:
    import astropy
except:
    !pip install --user astropy
    import astropy
import astropy.convolution
try:
    import pykrige
except:
    !pip install --user pykrige
    import pykrige
try:
    import pysheds
except:
    !pip install pysheds
    import pysheds
from pysheds.grid import Grid
try:
    from affine import Affine
except:
    !pip install affine
    from affine import Affine
try:
    import rasterio
except:
    !pip install rasterio
    import rasterio
try:
    import pyproj
except:
    !pip install pyproj
    import pyproj

#Download packages
codes_folder='/home/jovyan/codes'
project_dir=os.getcwd()
try:
    os.chdir(codes_folder)
except:
    os.mkdir(codes_folder)
    os.chdir(codes_folder)

if not os.path.exists(codes_folder+'/adore-doris/'):
  !git clone https://github.com/bosmanoglu/adore-doris.git
os.sys.path.append(codes_folder+'/adore-doris/lib/python')
os.sys.path.append(codes_folder)
os.chdir(project_dir) #Go back to project folder

#import modules after downloads
import gis
import surface_from_grad as sfg
import deramp as surface
import invdisttree
from tqdm.notebook import tqdm

#%matplotlib notebook

In [2]:
# Define convenience functions
def bounding_box_inside_bounding_box(small, big):
    s0=np.array([p[0] for p in small])
    s1=np.array([p[1] for p in small])
    b0=np.array([p[0] for p in big])
    b1=np.array([p[1] for p in big])
    inside=True
    if s0.min()<b0.min():
        inside=False
    if s0.max()>b0.max():
        inside=False
    if s1.min()<b1.min():
        inside=False
    if s1.max()>b1.max():
        inside=False
    return inside

def getGeoTransform(filename):
    warnings.warn("getGeoTransform will be deprecated in the future. Please use read_data instead.", PendingDeprecationWarning)
    return get_geotransform(filename)

def get_geotransform(filename):
    '''
    [top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution]=getGeoTransform('/path/to/file')
    '''
    #http://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file
    ds = gdal.Open(filename)
    return ds.GetGeoTransform()

def build_vrt(filename, input_file_list):
    vrt_options = gdal.BuildVRTOptions(resampleAlg='near', resolution='highest', separate=False, targetAlignedPixels=True)
    gdal.BuildVRT(filename,input_file_list,options=vrt_options)

def get_tiff_paths(paths):
    tiff_paths = !ls $paths | sort -t_ -k5,5
    return tiff_paths

def get_size(filename):
    """(width, height) = get_size(filename)
    """
    ds = gdal.Open(filename)
    width = ds.RasterXSize
    height = ds.RasterYSize
    ds=None
    return (width, height)
def get_proj4(filename):
    f=rasterio.open(filename)
    return pyproj.Proj(f.crs, preserve_units=True)  #used in pysheds
def clip_gT(gT, xmin, xmax, ymin, ymax, method='image'):
    '''calculate new geotransform for a clipped raster either using pixels or projected coordinates.
    clipped_gT=clip_gT(gT, xmin, xmax, ymin, ymax, method='image')
    method: 'image' | 'coord'
    '''
    if method == 'image':
      y,x=xy2coord(ymin, xmin, gT); #top left, reference, coordinate
    if method == 'coord':
      #find nearest pixel
      yi, xi = coord2xy(ymin, xmin, gT)
      #get pixel coordinate
      y,x=xy2coord(yi, xi, gT)
    gTc=list(gT)
    gTc[0]=y
    gTc[3]=x
    return tuple(gTc)
def xy2coord(x,y,gT):
    '''
    lon,lat=xy2coord(x,y,geoTransform)
    projects pixel index to position based on geotransform.
    '''
    coord_x=gT[0] + x*gT[1] + y*gT[2]
    coord_y=gT[3] + x*gT[4] + y*gT[5]
    return coord_x, coord_y

def coord2xy(x,y,gT):
    '''
    x,y = coord2xy(lon, lat, geoTransform)
    calculates pixel index closest to the lon, lat.
    '''
    #ref: https://gis.stackexchange.com/questions/221292/retrieve-pixel-value-with-geographic-coordinate-as-input-with-gdal/221430
    xOrigin = gT[0]
    yOrigin = gT[3]
    pixelWidth = gT[1]
    pixelHeight = -gT[5]

    col = np.array((x - xOrigin) / pixelWidth).astype(int)
    row = np.array((yOrigin - y) / pixelHeight).astype(int)

    return row,col
def fitSurface(x,y,z,X,Y):
    p0=[0,0.1,0.1]
    fitfunc = lambda p, x, y: p[0]+p[1]*x+p[2]*y
    errfunc = lambda p, x, y, z: abs(fitfunc(p,x,y) - z)
    planefit, success=optimize.leastsq(errfunc, p0, args=(x,y,z))
    return fitfunc(planefit, X,Y)    
def nonan(A, rows=False):
    if rows:
        return A[np.isnan(A).sum(1)==0];
    else:
        return A[~np.isnan(A)];

def bounding_box(filename, t_srs=None):
    """
    ((lon1,lat1), (lon2,lat2), (lon3,lat3), (lon4,lat4))=bounding_box('/path/to/file', t_srs=None) #returns x,y in native coordinate system
    ((lon1,lat1), (lon2,lat2), (lon3,lat3), (lon4,lat4))=bounding_box('/path/to/file', t_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')
    """
    gT=getGeoTransform(filename)
    width, height=get_size(filename)
    pts=(xy2coord(0,0,gT), xy2coord(width,0,gT), xy2coord(width, height,gT), xy2coord(0, height,gT))
    if t_srs is None:
        return pts
    else:
        pts_tsrs=[]
        s_srs=get_projection(filename, out_format='proj4')
        for p in pts:
            pts_tsrs.append(transformPoint(p[0], p[1], 0, s_srs=s_srs, t_srs=t_srs))
    return tuple(pts_tsrs)   
    
def get_waterbody(filename):

    corners=bounding_box(filename)

    west= corners[0][0]
    east= corners[1][0]
    south= corners[2][1]
    north= corners[0][1]

    cwd = os.getcwd()
    sw_path = f"{cwd}/S_WATER"
    if not os.path.exists(sw_path):
        os.mkdir(sw_path)

    lon = np.floor(west/10)
    lon=int(abs(lon*10))
    lat = np.ceil(north/10)
    lat=int(abs(lat*10))

    if (west<0 and north<0):
        urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon}W_{lat}S_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon}W_{lat}S.tif")
        if (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon-10}W_{lat}S_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon-10}W_{lat}S.tif")
        if (np.floor(north/10) != np.floor(south/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon}W_{lat+10}S_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon}W_{lat+10}S.tif")
        if (np.floor(north/10) != np.floor(south/10)) and (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon-10}W_{lat+10}S_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon-10}W_{lat+10}S.tif")
        print(f"lon: {lon}-{lon-10}W, lat: {lat}-{lat+10}S ")

    elif (west<0 and north>=0):
        urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon}W_{lat}N_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon}W_{lat}N.tif")
        if (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon-10}W_{lat}N_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon-10}W_{lat}N.tif")
        if (np.floor(north/10) != np.floor(south/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon}W_{lat-10}N_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon}W_{lat-10}N.tif")
        if (np.floor(north/10) != np.floor(south/10)) and (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon-10}W_{lat-10}N_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon-10}W_{lat-10}N.tif")
        print(f"lon: {lon}-{lon-10}W, lat: {lat}-{lat-10}N ")


    elif (west>=0 and north<0):
        urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon}E_{lat}S_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon}E_{lat}S.tif")
        if (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon+10}E_{lat}S_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon+10}E_{lat}S.tif")
        if (np.floor(north/10) != np.floor(south/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon}E_{lat+10}S_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon}E_{lat+10}S.tif")
        if (np.floor(north/10) != np.floor(south/10)) and (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon+10}E_{lat+10}S_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon+10}E_{lat+10}S.tif")
        print(f"lon: {lon}-{lon+10}E, lat: {lat}-{lat+10}S ")

    else:
        urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon}E_{lat}N_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon}E_{lat}N.tif")
        if (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon+10}E_{lat}N_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon+10}E_{lat}N.tif")
        if (np.floor(north/10) != np.floor(south/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon}E_{lat-10}N_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon}E_{lat-10}N.tif")
        if (np.floor(north/10) != np.floor(south/10)) and (np.floor(west/10) != np.floor(east/10)):
            urllib.request.urlretrieve(f"https://storage.googleapis.com/global-surface-water/downloads2/occurrence/occurrence_{lon+10}E_{lat-10}N_v1_1.tif", f"{cwd}/S_WATER/surface_water_{lon+10}E_{lat-10}N.tif")
        print(f"lon: {lon}-{lon+10}E, lat: {lat}-{lat-10}N ")


    # Building the virtual raster for Change Detection product(tiff)
    product_wpath = f"{cwd}/S_WATER/surface_water*.tif"
    #wildcard_path = f"{cwd}/change_VV_20170818T122205_20170830T122203.tif"
    print(product_wpath)
    os.chdir(cwd)

    get_ipython().system('gdalbuildvrt surface_water_map.vrt $product_wpath')

    #Clipping/Resampling Surface Water Map for AOI
    dim =get_size(filename)
    cmd_resamp=f"gdalwarp -te {west} {south} {east} {north} -ts {dim[0]} {dim[1]} -r lanczos {cwd}/surface_water_map.vrt {cwd}/surface_water_map_clip.vrt"
    print(cmd_resamp)
    os.system(cmd_resamp)

    #load resampled water map
    wimage_file =f"{cwd}/surface_water_map_clip.vrt"
    #water_map = gdal.Open(wimage_file)
    #print(f"X-dimension: {water_map.RasterXSize} Y-dimension: {water_map.RasterYSize}")

    #swater_map = water_map.ReadAsArray()
    swater_map =gis.readData(wimage_file)
    wmask=swater_map>0   # for small streams it shouldn't be over 30
    return wmask

def yesno(yes_no_question="[y/n]"):
    while True:
        # raw_input returns the empty string for "enter"
        yes = {'yes','y', 'ye'}
        no = {'no','n'}

        choice = input(yes_no_question+"[y/n]").lower()
        if choice in yes:
            return True
        elif choice in no:
            return False
        else:
            print("Please respond with 'yes' or 'no'")

In [3]:
class PathSelector():
    """
    Displays a file selection tree. Any file can be selected. 
    Selected path can be obtained by: PathSelector.accord.get_title(0)
    """
    def __init__(self,start_dir,select_file=True):
        self.file        = None 
        self.select_file = select_file
        self.cwd         = start_dir
        self.select      = ui.SelectMultiple(options=['init'],value=(),rows=10,description='') 
        self.accord      = ui.Accordion(children=[self.select]) 

        self.accord.selected_index = None # Start closed (showing path only)
        self.refresh(self.cwd)
        self.select.observe(self.on_update,'value')

    def on_update(self,change):
        if len(change['new']) > 0:
            self.refresh(change['new'][0])

    def refresh(self,item):
        path = os.path.abspath(os.path.join(self.cwd,item))

        if os.path.isfile(path):
            if self.select_file:
                self.accord.set_title(0,path)  
                self.file = path
                self.accord.selected_index = None
            else:
                self.select.value = ()

        else: # os.path.isdir(path)
            self.file = None 
            self.cwd  = path

            # Build list of files and dirs
            keys = ['[..]']; 
            for item in os.listdir(path):
                if item[0] == '.':
                    continue
                elif os.path.isdir(os.path.join(path,item)):
                    keys.append('['+item+']'); 
                else:
                    keys.append(item); 

            # Sort and create list of output values
            keys.sort(key=str.lower)
            vals = []
            for k in keys:
                if k[0] == '[':
                    vals.append(k[1:-1]) # strip off brackets
                else:
                    vals.append(k)

            # Update widget
            self.accord.set_title(0,path)  
            self.select.options = list(zip(keys,vals)) 
            with self.select.hold_trait_notifications():
                self.select.value = ()

<br><br>
<font face="Calibri" size="4"><b>Define some common parameters</b> </font>

In [4]:
#parameters setup
version="0.1.3"
water_classes = [1] # 1 has to be a water class, 0 is no water 
pattern="S1*_water_mask_combined.tif"
show_plots=False #turn this off for debugging with IPDB
water_level_sigma=2 #use 2*std to estimate max. water height (water level) for each object 
multi_processor=False
if show_plots:
    %matplotlib notebook

<br>
<font face="Calibri" size="4"><b>Enter the path to the directory holding your tiffs:</b> </font>

In [5]:
if yesno("Are you working with a single file?"):
    print("Choose your GDAL compatible Classified water extent file using the file browser below:")
    file_folder_func=lambda x: x
    single_file=True
else:
    print(f"Choose one of the water extent files inside the folder. All files with matching the following will be processed: {pattern}")
    file_folder_func=lambda x: os.path.dirname(x)
    single_file=False
f = PathSelector('.')
display(f.accord)

Are you working with a single file?[y/n]n
Choose one of the water extent files inside the folder. All files with matching the following will be processed: S1*_water_mask_combined.tif


Accordion(children=(SelectMultiple(options=(('0915.tar', '0915.tar'), ('0916.tar', '0916.tar'), ('[..]', '..')…

In [6]:
#Check if folder or file
tiff_path=file_folder_func(f.accord.get_title(0))
if not single_file:
    #merge all tifs
    tiffs=glob.glob(f"{tiff_path}/S1*_water_mask_combined.tif")
    print("Processing the following files:")
    pprint.pprint(tiffs)
    combined_vrt=os.path.join(tiff_path, os.path.basename(tiff_path)+'.vrt')
    combined_tif=os.path.join(tiff_path, os.path.basename(tiff_path)+'.tif')
    build_vrt(combined_vrt, tiffs)
    #translate vrt to tif. There is a problem warping with the vrt.
    cmd_translate=f"gdal_translate -of GTiff {combined_vrt} {combined_tif}"
    print(cmd_translate)
    os.system(cmd_translate)
    tiff_path=combined_tif
else:
    pass # do nothing. 

Processing the following files:
['/home/jovyan/FW_map/TS_Sally/input/S1B_IW_20200916T115757_DVR_RTC30_G_gpufed_60BF_water_mask_combined.tif',
 '/home/jovyan/FW_map/TS_Sally/input/S1B_IW_20200915T235417_DVR_RTC30_G_gpufed_AAC5_water_mask_combined.tif',
 '/home/jovyan/FW_map/TS_Sally/input/S1B_IW_20200915T235352_DVR_RTC30_G_gpufed_BF97_water_mask_combined.tif',
 '/home/jovyan/FW_map/TS_Sally/input/S1B_IW_20200915T235302_DVR_RTC30_G_gpufed_A44C_water_mask_combined.tif',
 '/home/jovyan/FW_map/TS_Sally/input/S1B_IW_20200916T115732_DVR_RTC30_G_gpufed_81D2_water_mask_combined.tif',
 '/home/jovyan/FW_map/TS_Sally/input/S1B_IW_20200916T115703_DVR_RTC30_G_gpufed_BED0_water_mask_combined.tif',
 '/home/jovyan/FW_map/TS_Sally/input/S1B_IW_20200915T235327_DVR_RTC30_G_gpufed_D876_water_mask_combined.tif']
gdal_translate -of GTiff /home/jovyan/FW_map/TS_Sally/input/input.vrt /home/jovyan/FW_map/TS_Sally/input/input.tif


<br>
<font face="Calibri" size="4"><b>Reproject tiffs from UTM to EPSG 4326:</b> </font>

In [8]:
print("Choose your GDAL compatible precalculated HAND file using the file browser below:")
f = PathSelector('.')
display(f.accord)        

Choose your GDAL compatible precalculated HAND file using the file browser below:


Accordion(children=(SelectMultiple(options=(('[..]', '..'), ('[S_WATER]', 'S_WATER'), ('clip_HAND5_TS_Sally_S1…

In [7]:
#checking current coordinate reference system
#tiff_path=f.accord.get_title(0)
tiff_dir = os.path.dirname(tiff_path)
info = (gdal.Info(tiff_path, options = ['-json']))
info = (json.loads(info))['coordinateSystem']['wkt']
utm = info.split('ID')[-1].split(',')[1][0:-2]
print(f"UTM Zone: {utm}")

#Reprojecting coordinate system
if utm != "4326":
    filename=tiff_path.split(tiff_dir)[1].split("/")[1]
    filenoext=os.path.splitext(filename)[0] #given vrt we want to force geotif output with tif extension
    cmd_reproj=f"gdalwarp -overwrite -s_srs EPSG:{utm} -t_srs EPSG:4326 -r cubicspline -of GTiff {tiff_dir}/{filename} {tiff_dir}/reproj_{filenoext}.tif"
    print(cmd_reproj)
    os.system(cmd_reproj)
else:
    os.symlink(f'{tiff_dir}/{filename}', f'{tiff_dir}/reproj_{filename}')
    
# Building the virtual raster for Change Detection product(tiff)
reprojected_flood_mask = f"{tiff_dir}/reproj_{filenoext}.tif"
print(f"Reprojected Flood Mask File: {reprojected_flood_mask}")
os.chdir(tiff_dir)

pixels, lines=get_size(reprojected_flood_mask)
print(f"X-dimension: {pixels} Y-dimension: {lines}")

UTM Zone: 32616
gdalwarp -overwrite -s_srs EPSG:32616 -t_srs EPSG:4326 -r cubicspline -of GTiff /home/jovyan/FW_map/TS_Sally/input/input.tif /home/jovyan/FW_map/TS_Sally/input/reproj_input.tif
Reprojected Flood Mask File: /home/jovyan/FW_map/TS_Sally/input/reproj_input.tif
X-dimension: 19264 Y-dimension: 21446


In [9]:
#checking extent of the map
info = (gdal.Info(reprojected_flood_mask, options = ['-json']))
info = (json.loads(info))['wgs84Extent']['coordinates']
#The bounding coordinates of the Flood Extent Product
west= info[0][0][0]
east= info[0][2][0]
south= info[0][2][1]
north= info[0][0][1]
print(f"Retrieved Extent of Flood Extent (w/e/s/n):{west}, {east}, {south}, {north}")

Retrieved Extent of Flood Extent (w/e/s/n):-91.6376785, -86.0900322, 27.882909, 34.0589277


In [10]:
#Check if HAND is valid. 
hand_dem=f.accord.get_title(0)
hand_dem_bb=bounding_box(hand_dem)
if not bounding_box_inside_bounding_box(bounding_box(reprojected_flood_mask), hand_dem_bb):
    print('Flood Extent Bounding Box:')
    print(bounding_box(reprojected_flood_mask))
    print('HAND bounding box:')
    print(hand_dem_bb)
    print('You can use BIG HAND Notebook to calculate HAND from a DEM.')
    raise ValueError('Image is not completely covered inside given HAND.')

#Clip HAND to the same size as the reprojected_flood_mask
filename=os.path.basename(hand_dem)
cmd_clip=f"gdalwarp -overwrite -te {west} {south} {east} {north} -ts {pixels} {lines} -r lanczos  -of GTiff {hand_dem} {tiff_dir}/clip_{filename}"
print(cmd_clip)
os.system(cmd_clip)
hand_array=gis.readData(f"{tiff_dir}/clip_{filename}")
if np.all(hand_array==0):
    print('HAND is all zeros. HAND DEM does not cover the imaged area.')
    raise ValueError # THIS SHOULD NEVER HAPPEN now that we are checking the bounding box. Unless HAND is bad.     

gdalwarp -overwrite -te -91.6376785 27.882909 -86.0900322 34.0589277 -ts 19264 21446 -r lanczos  -of GTiff /home/jovyan/FW_map/TS_Sally/HAND5_TS_Sally_S1.tif /home/jovyan/FW_map/TS_Sally/input/clip_HAND5_TS_Sally_S1.tif


<hr>
<font face="Calibri" size="5"><b>Generating Flood Mask</b> </font>
<br>

<font face="Calibri" size="4"><b>Flood Mask from Surface Water Layer: Downloading Global Surface Water Data </b></font>
<hr>
<font face="Calibri" size="3">All Global Surface Water data is produced under the Copernicus Programme: Jean-Francois Pekel, Andrew Cottam, Noel Gorelick, Alan S. Belward, High-resolution mapping of global surface water and its long-term changes. Nature 540, 418-422 (2016). (doi:10.1038/nature20584)</font>

In [11]:
#Get known Water Mask
known_water_mask=get_waterbody(reprojected_flood_mask)

lon: 100-90W, lat: 40-30N 
/home/jovyan/FW_map/TS_Sally/input/S_WATER/surface_water*.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -te -91.63767854730504 27.88290901508585 -86.09003218562131 34.05892769618372 -ts 19264 21446 -r lanczos /home/jovyan/FW_map/TS_Sally/input/surface_water_map.vrt /home/jovyan/FW_map/TS_Sally/input/surface_water_map_clip.vrt


<br>
<font face="Calibri" size="4"><b>Flood Mask from Hyp3-generated Change Detection product</b> </font>

In [12]:
#load and display change detection product from Hyp3
hyp_map = gdal.Open(reprojected_flood_mask)
change_map = hyp_map.ReadAsArray()

#Initial mask layer generation
for c in water_classes: # This allows more than a single water_class to be included in flood mask
    change_map[change_map==c]=1


mask=change_map==1
flood_mask=np.bitwise_or(mask,known_water_mask) #add known water mask... #Added 20200921

#flood_mask_filled=ndimage.morphology.binary_fill_holes(mask, structure=np.ones((10,10)))#REMOVED on 20200922
#mask=morphology.remove_small_objects(mask, min_size=1000, connectivity=1) #REMOVED 20200921
#flood_mask_filled_all=ndimage.morphology.binary_fill_holes(ndimage.morphology.binary_dilation(mask, iterations=1))#REMOVED 20200921

#Smoothing SAR flood masking
#flood_mask_filled_all=ndimage.morphology.binary_fill_holes(ndimage.morphology.binary_dilation(mask, iterations=5))
if show_plots:
    pl.matshow(flood_mask);pl.title('Final Flood Mask')



<br>
<font face="Calibri" size="4"> <b>Interpolating Masked Areas & Generating FD Map:</b> </font>

In [13]:
# Calculate Flood Depth - Show Progress Bar
flood_mask_labels, num_labels=ndimage.label(flood_mask)
print(f'Detected {num_labels} water bodies...')
object_slices=ndimage.find_objects(flood_mask_labels)
if show_plots:
    pl.matshow(flood_mask_labels);pl.colorbar()

flood_depth=np.zeros(flood_mask.shape)
#for k, osl in tqdm(enumerate(object_slices)): #Skip first, largest label.
for l in tqdm(range(1,num_labels)):
    # ids = np.argwhere(flood_mask_labels==l)
    # min0=max(ids[:,0].min()-1, 0)
    # max0=min(ids[:,0].max()+1, flood_mask_labels.shape[0])
    # min1=max(ids[:,1].min()-1, 0)
    # max1=min(ids[:,1].max()+1, flood_mask_labels.shape[1])
    slices=object_slices[l-1] #osl label=1 is in object_slices[0]
    min0=slices[0].start
    max0=slices[0].stop
    min1=slices[1].start
    max1=slices[1].stop
    flood_mask_labels_clip=flood_mask_labels[min0:max0, min1:max1] 
    #(values,counts) = np.unique(flood_mask_labels[osl][np.nonzero(flood_mask_labels[osl])],return_counts=True)
    #l=k+1#values[np.argmax(counts)]        
    #l=int(np.median(flood_mask_labels_clip))
    #if k+1!=l:
    #    print(f"k+1/l: {k+1}/{l}")
    #set_trace()
    flood_mask_clip=flood_mask[min0:max0, min1:max1].copy()
    flood_mask_clip[flood_mask_labels_clip!=l]=0 #Maskout other flooded areas (labels)
    hand_clip=hand_array[min0:max0, min1:max1]        
    m=np.nanmean(hand_clip[flood_mask_labels_clip==l])
    s=np.nanstd( hand_clip[flood_mask_labels_clip==l])
    flood_depth_clip=flood_depth[min0:max0, min1:max1]
    water_height=m+water_level_sigma*s
    flood_depth_clip[flood_mask_labels_clip==l]=water_height-hand_clip[flood_mask_labels_clip==l]    

if show_plots:
    pl.matshow(flood_depth);pl.colorbar();pl.clim([-5,5]);pl.title('Estimated Flood Depth')

Detected 518930 water bodies...


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: Mean of empty slice
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


<br>
<font face="Calibri" size="4"> <b>Export Results as Geotiff:</b> </font>

In [14]:
#Saving Estimated FD to geotiff
gT = gis.getGeoTransform(reprojected_flood_mask)

outfilename = tiff_path.split(tiff_dir)[1].split("/")[1]

gis.writeTiff(flood_depth, gT, filename=f"HAND_WaterDepth_{version}_{outfilename}", nodata=0, options = ["TILED=YES","COMPRESS=LZW","INTERLEAVE=BAND","BIGTIFF=YES"])
gis.writeTiff(flood_mask, gT, filename=f"Flood_mask_{version}_{outfilename}", options = ["TILED=YES","COMPRESS=LZW","INTERLEAVE=BAND","BIGTIFF=YES"])

flood_mask[known_water_mask]=0
flood_depth[np.bitwise_not(flood_mask)]=0
gis.writeTiff(flood_depth, gT, filename=f"HAND_FloodDepth_{outfilename}", nodata=0, options = ["TILED=YES","COMPRESS=LZW","INTERLEAVE=BAND","BIGTIFF=YES"])
print('Export complete.')

File written to: HAND_WaterDepth_0.1.3_input.tif
File written to: Flood_mask_0.1.3_input.tif
File written to: HAND_FloodDepth_input.tif
Export complete.


In [ ]:
#clear some intermediate files
os.remove(reprojected_flood_mask)
os.remove(f'{tiff_dir}/clip_{filename}')